In [ ]:
# Multinomial Naive Bayes

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import operator

In [7]:
#Training label
train_label = open('datasets/20news-bydate/matlab/train.label')

#pi is the fraction of each class
pi = {}

#Set a class index for each document as key
for i in range(1,21):
    pi[i] = 0

#Extract values from training labels
lines = train_label.readlines()

#Get total number of documents
total = len(lines)

#Count the occurence of each class
for line in lines:
    val = int(line.split()[0])
    pi[val] += 1

#Divide the count of each class by total documents
for key in pi:
    pi[key] /= total

print("Probability of each class:")
print("\n".join("{}: {}".format(k, v) for k, v in pi.items()))

Probability of each class:
1: 0.04259472890229834
2: 0.05155736977549028
3: 0.05075871860857219
4: 0.05208980388676901
5: 0.051024935664211554
6: 0.052533498979501284
7: 0.051646108794036735
8: 0.052533498979501284
9: 0.052888455053687104
10: 0.0527109770165942
11: 0.05306593309078002
12: 0.0527109770165942
13: 0.05244475996095483
14: 0.0527109770165942
15: 0.052622237998047744
16: 0.05315467210932647
17: 0.04836276510781791
18: 0.05004880646020055
19: 0.04117490460555506
20: 0.033365870973467035


In [8]:
#Training data
train_data = open('datasets/20news-bydate/matlab/train.data')
df = pd.read_csv(train_data, delimiter=' ', names=['docIdx', 'wordIdx', 'count'])

#Training label
label = []
train_label = open('datasets/20news-bydate/matlab/train.label')
lines = train_label.readlines()
for line in lines:
    label.append(int(line.split()[0]))

#Increase label length to match docIdx
docIdx = df['docIdx'].values
i = 0
new_label = []
for index in range(len(docIdx)-1):
    new_label.append(label[i])
    if docIdx[index] != docIdx[index+1]:
        i += 1
new_label.append(label[i]) #for-loop ignores last value

#Add label column
df['classIdx'] = new_label

df.head()

,docIdx,wordIdx,count,classIdx
0,1,1,4,1
1,1,2,2,1
2,1,3,10,1
3,1,4,4,1
4,1,5,2,1


In [9]:
#Alpha value for smoothing
a = 0.001

#Calculate probability of each word based on class
pb_ij = df.groupby(['classIdx','wordIdx'])
pb_j = df.groupby(['classIdx'])
Pr =  (pb_ij['count'].sum() + a) / (pb_j['count'].sum() + 16689)

#Unstack series
Pr = Pr.unstack()

#Replace NaN or columns with 0 as word count with a/(count+|V|+1)
for c in range(1,21):
    Pr.loc[c,:] = Pr.loc[c,:].fillna(a/(pb_j['count'].sum()[c] + 16689))

#Convert to dictionary for greater speed
Pr_dict = Pr.to_dict()

Pr


wordIdx,1,2,3,4,5,6,7,8,9,10,...,53966,53967,53968,53969,53970,53971,53972,53973,53974,53975
classIdx,,,,,,,,,,,,,,,,,,,,,
1,7.855542e-05,0.000381,1.661627e-03,5.438638e-05,0.000495,0.000248,3.625960e-05,6.048302e-06,0.000205,8.459224e-04,...,6.042260e-09,6.042260e-09,6.042260e-09,6.042260e-09,6.042260e-09,6.042260e-09,6.042260e-09,6.042260e-09,6.042260e-09,6.042260e-09
2,4.722740e-04,0.000464,7.871103e-09,1.338166e-04,0.000110,0.000457,7.871890e-05,4.723449e-05,0.001354,2.362118e-05,...,7.871103e-09,7.871103e-09,7.871103e-09,7.871103e-09,7.871103e-09,7.871103e-09,7.871103e-09,7.871103e-09,7.871103e-09,7.871103e-09
3,1.023768e-04,0.000642,9.306135e-09,1.582136e-04,0.000195,0.000316,1.862158e-05,1.862158e-05,0.001340,9.306135e-09,...,9.306135e-09,9.306135e-09,9.306135e-09,9.306135e-09,9.306135e-09,9.306135e-09,9.306135e-09,9.306135e-09,9.306135e-09,9.306135e-09
4,6.907239e-05,0.000268,8.632969e-09,8.632969e-09,0.000086,0.000414,1.727457e-05,8.641602e-06,0.000414,8.632969e-09,...,8.632969e-09,8.632969e-09,8.632969e-09,8.632969e-09,8.632969e-09,8.632969e-09,8.632969e-09,8.632969e-09,8.632969e-09,8.632969e-09
5,5.833066e-05,0.000321,9.720157e-09,9.729877e-06,0.000010,0.000457,9.729877e-06,9.720157e-09,0.000457,9.720157e-09,...,9.720157e-09,9.720157e-09,9.720157e-09,9.720157e-09,9.720157e-09,9.720157e-09,9.720157e-09,9.720157e-09,9.720157e-09,9.720157e-09
6,2.772348e-04,0.001309,5.898487e-09,4.659864e-04,0.000088,0.000307,1.238741e-04,1.770136e-05,0.001398,5.898487e-09,...,5.898487e-09,5.898487e-09,5.898487e-09,5.898487e-09,5.898487e-09,5.898487e-09,5.898487e-09,5.898487e-09,5.898487e-09,5.898487e-09
7,1.285628e-08,0.000360,1.285628e-08,2.572542e-05,0.000026,0.000411,1.285628e-08,1.285628e-08,0.000360,3.858170e-05,...,1.285628e-08,1.285628e-08,1.285628e-08,1.285628e-08,1.285628e-08,1.285628e-08,1.285628e-08,1.285628e-08,1.285628e-08,1.285628e-08
8,6.881972e-05,0.000413,7.645786e-09,7.645786e-09,0.000099,0.000658,5.352815e-05,2.294500e-05,0.000138,7.645786e-09,...,7.645786e-09,7.645786e-09,7.645786e-09,7.645786e-09,7.645786e-09,7.645786e-09,7.645786e-09,7.645786e-09,7.645786e-09,7.645786e-09
9,1.173399e-04,0.000562,8.380825e-09,3.353168e-05,0.000034,0.000696,2.515085e-05,8.389205e-06,0.000034,8.380825e-09,...,8.380825e-09,8.380825e-09,8.380825e-09,8.380825e-09,8.380825e-09,8.380825e-09,8.380825e-09,8.380825e-09,8.380825e-09,8.380825e-09


In [11]:
#Common stop words from online
stop_words = [
"a", "about", "above", "across", "after", "afterwards",
"again", "all", "almost", "alone", "along", "already", "also",
"although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "as", "at", "be", "became", "because", "become","becomes", "becoming", "been", "before", "behind", "being", "beside", "besides", "between", "beyond", "both", "but", "by","can", "cannot", "cant", "could", "couldnt", "de", "describe", "do", "done", "each", "eg", "either", "else", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "find","for","found", "four", "from", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "i", "ie", "if", "in", "indeed", "is", "it", "its", "itself", "keep", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mine", "more", "moreover", "most", "mostly", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next","no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part","perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "she", "should","since", "sincere","so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "take","than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they",
"this", "those", "though", "through", "throughout",
"thru", "thus", "to", "together", "too", "toward", "towards",
"under", "until", "up", "upon", "us",
"very", "was", "we", "well", "were", "what", "whatever", "when",
"whence", "whenever", "where", "whereafter", "whereas", "whereby",
"wherein", "whereupon", "wherever", "whether", "which", "while",
"who", "whoever", "whom", "whose", "why", "will", "with",
"within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves"
]


In [14]:
vocab = open('datasets/20news-bydate/vocabulary.txt')
vocab_df = pd.read_csv(vocab, names = ['word'])
vocab_df = vocab_df.reset_index()
vocab_df['index'] = vocab_df['index'].apply(lambda x: x+1)
vocab_df.head()

,index,word
0,1,archive
1,2,name
2,3,atheism
3,4,resources
4,5,alt


In [15]:
#Index of all words
tot_list = set(vocab_df['index'])

#Index of good words
vocab_df = vocab_df[~vocab_df['word'].isin(stop_words)]
good_list = vocab_df['index'].tolist()
good_list = set(good_list)

#Index of stop words
bad_list = tot_list - good_list

#Set all stop words to 0
for bad in bad_list:
    for j in range(1,21):
        Pr_dict[j][bad] = a/(pb_j['count'].sum()[j] + 16689)


In [17]:
#Calculate IDF
tot = len(df['docIdx'].unique())
pb_ij = df.groupby(['wordIdx'])
IDF = np.log(tot/pb_ij['docIdx'].count())
IDF_dict = IDF.to_dict()
def MNB(df, smooth = False, IDF = False):
    '''
    Multinomial Naive Bayes classifier
    :param df [Pandas Dataframe]: Dataframe of data
    :param smooth [bool]: Apply Smoothing if True
    :param IDF [bool]: Apply Inverse Document Frequency if True
    :return predict [list]: Predicted class ID
    '''
    #Using dictionaries for greater speed
    df_dict = df.to_dict()
    new_dict = {}
    prediction = []

    #new_dict = {docIdx : {wordIdx: count},....}
    for idx in range(len(df_dict['docIdx'])):
        docIdx = df_dict['docIdx'][idx]
        wordIdx = df_dict['wordIdx'][idx]
        count = df_dict['count'][idx]
        try:
            new_dict[docIdx][wordIdx] = count
        except:
            new_dict[df_dict['docIdx'][idx]] = {}
            new_dict[docIdx][wordIdx] = count

    #Calculating the scores for each doc
    for docIdx in range(1, len(new_dict)+1):
        score_dict = {}
        #Creating a probability row for each class
        for classIdx in range(1,21):
            score_dict[classIdx] = 1
            #For each word:
            for wordIdx in new_dict[docIdx]:
                #Check for frequency smoothing
                #log(1+f)*log(Pr(i|j))
                if smooth:
                    try:
                        probability=Pr_dict[wordIdx][classIdx]
                        power = np.log(1+ new_dict[docIdx][wordIdx])
                        #Check for IDF
                        if IDF:
                            score_dict[classIdx]+=(
                               power*np.log(
                               probability*IDF_dict[wordIdx]))
                        else:
                            score_dict[classIdx]+=power*np.log(
                                                   probability)
                    except:
                        #Missing V will have log(1+0)*log(a/16689)=0
                        score_dict[classIdx] += 0
                #f*log(Pr(i|j))
                else:
                    try:
                        probability = Pr_dict[wordIdx][classIdx]
                        power = new_dict[docIdx][wordIdx]
                        score_dict[classIdx]+=power*np.log(
                                           probability)
                        #Check for IDF
                        if IDF:
                            score_dict[classIdx]+= power*np.log(
                                   probability*IDF_dict[wordIdx])
                    except:
                        #Missing V will have 0*log(a/16689) = 0
                        score_dict[classIdx] += 0
            #Multiply final with pi
            score_dict[classIdx] +=  np.log(pi[classIdx])

        #Get class with max probabilty for the given docIdx
        max_score = max(score_dict, key=score_dict.get)
        prediction.append(max_score)

    return prediction

In [19]:
regular_predict = MNB(df, smooth=False, IDF=False)
smooth_predict  = MNB(df, smooth=True, IDF=False)
tfidf_predict   = MNB(df, smooth=False, IDF=True)
all_predict     = MNB(df, smooth=True, IDF=True)#Get list of labels
train_label = pd.read_csv('datasets/20news-bydate/matlab/train.label', names=['t'])
train_label = train_label['t'].tolist()
total = len(train_label)
models = [regular_predict, smooth_predict,
          tfidf_predict, all_predict]
strings = ['Regular', 'Smooth', 'IDF', 'Both']

for m,s in zip(models,strings):
    val = 0
    for i,j in zip(m, train_label):
        if i != j:
            val +=1
        else:
            pass
    print(s,"Error:\t\t",val/total * 100, "%")

Regular Error:		 3.4608217233117404 %
Smooth Error:		 1.8546454876209069 %
IDF Error:		 3.478569527021031 %
Both Error:		 1.8546454876209069 %


In [21]:
#Get test data
test_data = open('datasets/20news-bydate/matlab/test.data')
df = pd.read_csv(test_data, delimiter=' ', names=['docIdx', 'wordIdx', 'count'])

#Get list of labels
test_label = pd.read_csv('datasets/20news-bydate/matlab/test.label', names=['t'])
test_label= test_label['t'].tolist()

#MNB Calculation
predict = MNB(df, smooth = True, IDF = False)

total = len(test_label)
val = 0
for i,j in zip(predict, test_label):
    if i == j:
        val +=1
    else:
        pass

print("Accuracy:\t",val/total * 100, "%")

Accuracy:	 77.42838107928048 %
